First mount your google drive to this collab script, in order to read in the dataset an other things

In [2]:
#Mount Drive

Next, we will import the necessary modules which are:

- pandas
- numpy
- matplotlib (pyplot)

In [3]:
#Import the modules using the 'import' command


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Import those nessesary to do the learning
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Take the data files and process them into a pandas dataframe

In [21]:
#TODO: Add the other data frames, and then combine. Also add collumn names
columnNames = ['Age', 'Sex', 'Chest Pain Type', 'Resting Blood Pressure', 'Cholesterol', 'Fasting Blood Sugar', 'Resting ECG', 'Max Heart Rate', 'Exercise Induced Angina', 'ST Depression', 'Slope of ST Peak', 'Number of Major Vessels', 'Thalassemia', 'Diagnosis']
heartDisease_DataFrame = pd.DataFrame(pd.concat([pd.read_csv('processed_cleveland.data', names=columnNames), pd.read_csv('processed_hungarian.data', names=columnNames), pd.read_csv('processed_switzerland.data', names=columnNames), pd.read_csv('processed_va.data', names=columnNames)])).reset_index().drop('index', axis=1)

# print(heartDisease_DataFrame.replace('?', np.nan).dropna().to_string())
print(heartDisease_DataFrame.replace('?', np.nan).to_string())
# heartDisease_DataFrame.head()

      Age  Sex  Chest Pain Type Resting Blood Pressure Cholesterol Fasting Blood Sugar Resting ECG Max Heart Rate Exercise Induced Angina ST Depression Slope of ST Peak Number of Major Vessels Thalassemia  Diagnosis
0    63.0  1.0              1.0                  145.0       233.0                 1.0         2.0          150.0                     0.0           2.3              3.0                     0.0         6.0          0
1    67.0  1.0              4.0                  160.0       286.0                 0.0         2.0          108.0                     1.0           1.5              2.0                     3.0         3.0          2
2    67.0  1.0              4.0                  120.0       229.0                 0.0         2.0          129.0                     1.0           2.6              2.0                     2.0         7.0          1
3    37.0  1.0              3.0                  130.0       250.0                 0.0         0.0          187.0                     0.